In [ ]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import warnings
from sklearn import metrics
from sklearn.metrics import * 

plt.rc('font', size=12)
warnings.filterwarnings('ignore')

In [ ]:
#Load data
#ESCAPE
escape = pd.read_csv("Preprocessed Data/EscapeAllData.csv", sep=",", index_col='DEIDNUM').sort_index() #all feature dataset
escapeSV = pd.read_csv("Preprocessed Data/EscapeAllDataSingleValue.csv", sep=",", index_col='DEIDNUM').sort_index() #all feature dataset
escapeLabels  = pd.read_csv("../Data/Original DataFrames/Labels.csv", sep=",", index_col='DEIDNUM').sort_index() #labels for prediction classes 

#BEST
#Only has discharge data + no hemo
best = pd.read_csv("Preprocessed Data/AllDataBEST.csv", sep=",", index_col='ID').sort_index() #all feature dataset
bestLabels  = pd.read_csv("Preprocessed Data/LabelsBEST.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 

#HF ACTION
#Only has discharge data + no hemo
hfaction = pd.read_csv("Preprocessed Data/AllDataHF-ACTION.csv", sep=",", index_col='ID').sort_index() #all feature dataset
hfactionLabels  = pd.read_csv("Preprocessed Data/LabelsHF-ACTION.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 

#Guide it
guide = pd.read_csv("Preprocessed Data/GUIDEAllData.csv", sep=",", index_col='ID').sort_index() #all feature dataset
guideSV = pd.read_csv("Preprocessed Data/GUIDEAllDataSingleValue.csv", sep=",", index_col='ID').sort_index() #all feature dataset
guideLabels  = pd.read_csv("Preprocessed Data/LabelsGUIDE-IT.csv", sep=",", index_col='ID').sort_index() #labels for prediction classes 


# #Cardiogenic shock
# cardShockHemo = pd.read_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/HemoCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
# cardShockAllData = pd.read_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/AllDataCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
# cardShockLabels = pd.read_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/LabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()
# cardShockHemoLabels = pd.read_csv("../Data Validation/Cardiogenic Shock/Original DataFrames/HemoLabelsCardiogenicShock.csv", sep=",", index_col='ID').sort_index()

# #Serial Cardiac
# serialHemo = pd.read_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/HemoSerialCardiac.csv", sep=",", index_col='ID').sort_index()
# serialAllData = pd.read_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/AllDataSerialCardiac.csv", sep=",", index_col='ID').sort_index()
# serialHemoLabels = pd.read_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/HemoLabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()
# serialLabels = pd.read_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/LabelsSerialCardiac.csv", sep=",", index_col='ID').sort_index()

# #HF-ACTION
# hfactionAllData = pd.read_csv("../Data Validation/HF-ACTION/Original DataFrames/AllDataHF-ACTION.csv", sep=",", index_col='ID').sort_index()
# hfactionLabels = pd.read_csv("../Data Validation/HF-ACTION/Original DataFrames/LabelsHF-ACTION.csv", sep=",", index_col='ID').sort_index()

# #BEST
# bestAllData = pd.read_csv("../Data Validation/BEST/Original DataFrames/AllDataBEST.csv", sep=",", index_col='ID').sort_index()
# bestLabels = pd.read_csv("../Data Validation/BEST/Original DataFrames/LabelsBEST.csv", sep=",", index_col='ID').sort_index()

# #Guide it
# guideAllData = pd.read_csv("../Data Validation/GUIDE-IT/Original DataFrames/AllDataGUIDE-IT.csv", sep=",", index_col='ID').sort_index()
# guideLabels = pd.read_csv("../Data Validation/GUIDE-IT/Original DataFrames/LabelsGUIDE-IT.csv", sep=",", index_col='ID').sort_index()



In [ ]:
cardShockAllData

In [ ]:
#Calculate ESCAPE Risk score

def calcESCAPEScore(data):
    lst = []
    for idx in data.index:
        pt = data.loc[idx]
        scr = 0
        if pt.iloc[0] > 70: #Age
            scr += 1
        if pt.iloc[1] > 40: #BUN
            scr += 1
        if pt.iloc[1] > 90: #BUN
            scr += 1
        if pt.iloc[2] < 300: #Walk ft
            scr += 1
        if pt.iloc[3] < 130: #SOD
            scr += 1
        if pt.iloc[4] == 1.0 or pt.iloc[5] == 1.0: #CPR or MEVT
            scr += 2
        if pt.iloc[6] > 240: #DIUR Dose
            scr += 1
        if pt.iloc[7] == 0.0: #BETA blockers
            scr += 1
        if pt.iloc[8] > 500: #BNP
            scr += 1
        if pt.iloc[8] > 1300: #BNP
            scr += 3
            
        #associate score with prob of outcome
        prob = 0
        if scr == 0:
            prob = 0.033
        elif scr == 1:
            prob = 0.065
        elif scr == 2:
            prob = 0.123
        elif scr == 3:
            prob = 0.223
        elif scr == 4:
            prob = 0.368
        elif scr == 5:
            prob = 0.543
        elif scr == 6:
            prob = 0.708
        elif scr == 7:
            prob = 0.831
        else: #>= 8 
            prob = 0.909

        lst.append(prob)


    rskDF = pd.DataFrame(lst, columns=['Score'], index=data.index)
    return rskDF

def calcEUROMACS(data): #Preoperative model
    lst = []
    for idx in data.index:
        pt = data.loc[idx]
        scr = 0
        
        if data.iloc[0] > 0.54 or data.iloc[1] > 0.54: #RAP or PCWP
            scr += 2
        if data.iloc[2] <= 10: #hemoglobin
            scr += 1
        if data.iloc[3] == 1.0: #multiple intravenous inotropes
            scr += 2.5
        if data.iloc[4] <= 3: #Intermacs class 1-3
            scr += 2
        if data.iloc[5] == 1.0: #severe RV dysfunction
            scr += 2
            
        lst.append(scr)
        
    rskDF = pd.DataFrame(lst, columns=['Score'], index=data.index)
    return rskDF
        
# def calcHealthABC(data):
#     lst = []
#     for idx in data.index:
#         pt = data.loc[idx]
#         scr = 0
        
#         if data.iloc[0] > 0.54 or data.iloc[1] > 0.54: #RAP or PCWP
#             scr += 2
#         if data.iloc[2] <= 10: #hemoglobin
#             scr += 1
#         if data.iloc[3] == 1.0: #multiple intravenous inotropes
#             scr += 2.5
#         if data.iloc[4] <= 3: #Intermacs class 1-3
#             scr += 2
#         if data.iloc[5] == 1.0: #severe RV dysfunction
#             scr += 2
            
#         lst.append(scr)
        
#     rskDF = pd.DataFrame(lst, columns=['Score'], index=data.index)
#     return rskDF



In [ ]:
aucs = []

#Calculate all scores for this dataset

#ESCAPE Risk score
esRs = escape.filter(['Age','BUN_D','SixFtWlk_D', 'SOD_D', 'CPR', 'MEVT', 'DIURDSE_D', 'BET_D', 'BNP_D'], axis=1)
res = calcESCAPEScore(esRs)
#given actual labels and predicted probability of the event (outcome), get AUC
actualVals = escapeLabels['Death']
scr = roc_auc_score(actualVals, res['Score'])
print("Escape Risk AUC:", scr)
aucs.append(scr)

#EUROMACS - TBD once figure out how to properly code ...
# esRs = escapeSV.filter(['RAP','PCWP','HEM', 'INOT', 'INTERMACS', 'SevRV'], axis=1)
# res = calcEUROMACS(data)


##
Intermacs --> 

Needed Vars:
CPR or MEVT
BNP
Diuretics dose
inotropes
intermacs?
severe rv dysfunction?

HF diagnosed in last 18 months
Diuretics doses:
- Furosemide
- Bumetanide
- Torsemide
- Metolazone
- ChloroTZ

HGB
uric acid
Total cholesterol


In [ ]:
#BEST has

Diuretics but no dose
inotropes
HGB
uric acid
Total cholesterol

In [ ]:
#HF action has

BNP
Diuretics dose

Diuretics doses:
- Furosemide

Total cholesterol


In [ ]:
#GUIDe it has
Needed Vars:
MEVT
BNP
Diuretics dose
inotropes

Diuretics doses - use furosemide from diurdse

uric acid
Total cholesterol


In [ ]:
actual = []
actual.extend([0.077 for i in range(91)])
actual.extend([0.104 for i in range(125)])
actual.extend([0.167 for i in range(114)])
actual.extend([0.264 for i in range(53)])
actual.extend([0.448 for i in range(29)])
actual.extend([0.800 for i in range(15)])
actual.extend([0.750 for i in range(4)])
actual.extend([1.000 for i in range(1)])
actual.extend([1.000 for i in range(1)])

predBNP = []
predBNP.extend([0.033 for i in range(91)])
predBNP.extend([0.065 for i in range(125)])
predBNP.extend([0.123 for i in range(114)])
predBNP.extend([0.223 for i in range(53)])
predBNP.extend([0.368 for i in range(29)])
predBNP.extend([0.543 for i in range(15)])
predBNP.extend([0.708 for i in range(4)])
predBNP.extend([0.831 for i in range(1)])
predBNP.extend([0.909 for i in range(1)])

predNoBNP = []
predNoBNP.extend([0.053 for i in range(91)])
predNoBNP.extend([0.103 for i in range(125)])
predNoBNP.extend([0.189 for i in range(114)])
predNoBNP.extend([0.322 for i in range(53)])
predNoBNP.extend([0.492 for i in range(29)])
predNoBNP.extend([0.664 for i in range(15)])
predNoBNP.extend([0.801 for i in range(4)])
predNoBNP.extend([0.891 for i in range(1)])
predNoBNP.extend([0.943 for i in range(1)])
